# Segmentación para SDV 

Este código se utiliza para segmentar los datos reales y prepararlos para ingresarlos al generdador de datos sintéticos SDV

In [1]:
#Carga de paquetes 
#Core
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import matplotlib.dates as md

In [3]:
#Ajuste del directorio - (Al trabajar con un disco externo se requiere ajustar el directorio dependiendo del ordenador en el que se trabaje)
#Si se trabaja desde el odenador propio correr esta celda
path = 'C:\\Users\\Vaneza'
os.chdir(path)
os.getcwd()
disco = 'E'

In [ ]:
#Si se trabaja desde el ordenador del trabajo correr esta celda
path = 'C:\\Users\\Y8764982H\\OneDrive - Generalitat de Catalunya/TFG/'
os.chdir(path)
os.getcwd()
disco = 'D'

In [18]:
memory_usage_psutil()

28.58203125

## Carga de tablas

In [4]:
files_2m = []
for dirname, _, filenames in os.walk('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\All_Tables2\\'%(disco)):
    for filename in filenames:
        if filename.find(".csv") != -1 and not "-checkpoint.csv" in filename:
            files_2m.append(filename)
tables_2m = {p: pd.read_csv('%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\All_Tables2\\'%(disco) + p) for p in files_2m}

print(tables_2m.keys())

dict_keys(['all_CT_pat_2m.csv', 'all_PD_pat_2m.csv', 'all_CT_pat_2m_NF.csv', 'all_PD_pat_2m_NF.csv'])


In [5]:
#Selección de Canales
desc = ['Patient','Diagnosis','Time']
#Channels = ['F7', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6']

# Channels = ['Fz', 'FC1', 'C3', 'T7','CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10',
#             'CP6', 'CP2', 'Cz', 'C4', 'FC2', 'F1', 'FC3', 'C1', 'C5','TP7', 'CP3', 'P1', 'P5', 'PO7',
#             'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2']

Channels = ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5',
       'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2',
       'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6',
       'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3',
       'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8',
       'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8',
       'AF4', 'F2', 'FCz']
print(len(Channels))

63


In [6]:
CT_Pat = tables_2m['all_CT_pat_2m.csv']
CT_Pat.columns

Index(['Patient', 'Diagnosis', 'Time', 'Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5',
       'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2',
       'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6',
       'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3',
       'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8',
       'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8',
       'AF4', 'F2', 'FCz'],
      dtype='object')

In [ ]:
# Seleccionar de cada tabla los canales que se definieron antes
CT_Pat = tables_2m['all_CT_pat_2m.csv'][desc + Channels]
CT_Pat_NF = tables_2m['all_CT_pat_2m_NF.csv'][desc + Channels]
PD_Pat = tables_2m['all_PD_pat_2m.csv'][desc + Channels]
PD_Pat_NF = tables_2m['all_PD_pat_2m_NF.csv'][desc + Channels]
CT_Pat

### Etapa de división

In [ ]:
#Creación de nombres de pacientes para crear ventanas de 30 segundos
for i in range(0,14):
    for n in range(1,5):
        CT = ['CT%s_%s'%(i,n)] * 15000
        PD = ['PD%s_%s'%(i,n)] * 15000
        if (i == 0) & (n == 1):
            CT1 = pd.DataFrame(CT,columns=['Patient'])
            PD1 = pd.DataFrame(PD,columns=['Patient'])
        else:
            CT1_t = pd.DataFrame(CT,columns=['Patient'])
            PD1_t = pd.DataFrame(PD,columns=['Patient'])
            CT1 = pd.concat([CT1,CT1_t],axis=0).reset_index(drop=True)
            PD1 = pd.concat([PD1,PD1_t],axis=0).reset_index(drop=True)
        
PD1

In [ ]:
# Asignar los pacientes de control a la tabla de control
DA_CT = CT_Pat.copy(deep=True).drop(columns='Patient').reset_index(drop=True)
DA_CT = pd.concat([CT1,DA_CT],axis=1)
DA_CT

In [ ]:
# Asignar los pacientes de párkinson a la tabla de párkinson
DA_PD = PD_Pat.copy(deep=True).drop(columns='Patient').reset_index(drop=True)
DA_PD = pd.concat([PD1,DA_PD],axis=1)
DA_PD

In [11]:
DA_CT['Patient'].nunique()

56

In [12]:
DA_PD['Patient'].unique()

array(['PD0_1', 'PD0_2', 'PD0_3', 'PD0_4', 'PD1_1', 'PD1_2', 'PD1_3',
       'PD1_4', 'PD2_1', 'PD2_2', 'PD2_3', 'PD2_4', 'PD3_1', 'PD3_2',
       'PD3_3', 'PD3_4', 'PD4_1', 'PD4_2', 'PD4_3', 'PD4_4', 'PD5_1',
       'PD5_2', 'PD5_3', 'PD5_4', 'PD6_1', 'PD6_2', 'PD6_3', 'PD6_4',
       'PD7_1', 'PD7_2', 'PD7_3', 'PD7_4', 'PD8_1', 'PD8_2', 'PD8_3',
       'PD8_4', 'PD9_1', 'PD9_2', 'PD9_3', 'PD9_4', 'PD10_1', 'PD10_2',
       'PD10_3', 'PD10_4', 'PD11_1', 'PD11_2', 'PD11_3', 'PD11_4',
       'PD12_1', 'PD12_2', 'PD12_3', 'PD12_4', 'PD13_1', 'PD13_2',
       'PD13_3', 'PD13_4'], dtype=object)

### Cambio de la columna de tiempo de 0 a 30 s

In [ ]:
rng = np.arange(0.000, 30, 0.002)
rng = pd.DataFrame(rng,columns=['Time'])
#print(rng)
#print(len(rng))
Time_df = rng
for i in range(0,55):
    Time_df = pd.concat([Time_df,rng],axis=0).reset_index(drop=True)
Time_df

In [ ]:
CT_Pat_new = DA_CT.copy(deep=True)
CT_Pat_new['Time'] = Time_df
CT_Pat_new

In [ ]:
PD_Pat_new = DA_PD.copy(deep=True)
PD_Pat_new['Time'] = Time_df
PD_Pat_new

In [16]:
#Comprobacion 
comprobacion = DA_PD[(DA_PD["Patient"] == "PD0_1")].reset_index(drop=True)
comprobacion

,Patient,Diagnosis,Time,Fp1,Fz,F3,F7,FT9,FC5,FC1,...,TP8,C6,C2,FC4,FT8,F6,AF8,AF4,F2,FCz
0,PD0_1,1,15000,26.854488,10.573020,8.470492,-19.501508,20.007938,30.291653,11.195200,...,-0.945368,-0.799274,-0.747911,22.221163,-11.274306,12.686849,-32.483212,11.547446,10.375781,6.106222
1,PD0_1,1,15002,19.209267,13.396275,9.310676,-30.357076,19.271278,23.147037,11.115347,...,-0.694866,-1.329778,-2.991693,12.711124,-5.869117,14.424196,-5.114361,17.730690,13.520765,5.032362
2,PD0_1,1,15004,10.735276,14.444188,10.508716,-32.118495,17.557070,11.267493,10.927211,...,-0.540682,-1.118885,-5.662233,-0.134446,0.775774,7.170654,31.776314,24.942233,14.765049,3.136412
3,PD0_1,1,15006,3.370448,13.748147,11.568609,-23.256991,15.637565,-3.186819,10.574400,...,-0.284498,-0.852348,-7.770909,-11.729151,6.750742,-5.321692,62.985687,28.733573,14.102434,1.012423
4,PD0_1,1,15008,-1.103081,12.000841,12.095011,-5.236588,14.316720,-16.642865,10.000229,...,0.279053,-0.834205,-8.484601,-17.889033,10.437040,-16.026592,75.268529,26.528978,12.249151,-0.673208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,PD0_1,1,44990,14.719839,19.516850,15.843049,-28.582582,-8.644778,28.952445,12.082195,...,-5.461334,9.460727,4.074403,-7.289758,12.222443,70.028404,-26.989012,45.261925,20.096544,6.456228
14996,PD0_1,1,44992,16.647831,22.946203,17.675892,-27.992986,-7.103919,30.965213,12.312153,...,-3.634701,7.757430,5.861971,-1.227213,5.841956,66.273832,-41.496070,29.828012,23.798298,7.169446
14997,PD0_1,1,44994,18.974968,24.321945,18.142325,-22.887623,-5.548256,28.303859,11.638433,...,-2.252548,4.245121,7.861901,9.463737,-0.718418,52.752531,-49.914512,11.070918,25.334423,7.823515
14998,PD0_1,1,44996,21.837524,23.743723,17.033716,-12.230357,-4.058296,19.601625,10.153971,...,-1.265764,0.163389,9.870524,23.131549,-5.064771,32.851888,-53.642972,-9.854916,24.783735,8.574391


### Cambio de la columna de tiempo de int a datetime

In [48]:
from datetime import datetime, date, timedelta
import ast

days = []
d = date(2000, 1, 1)
days.append(d)
for i in range(0,14999):
    time = ast.literal_eval(str(2))
    d = d + timedelta(days=time)
    days.append(d)
    
days = pd.DataFrame(days)
days.columns= ['Time']

#days = pd.concat([days,days,days,days,days,days,days,days,days,days,days,days,days,days],axis=0).reset_index(drop=True)
days_df = days
for h in range(0,55):
    days_df = pd.concat([days_df,days],axis=0).reset_index(drop=True)
days_df

,Time
0,2000-01-01
1,2000-01-03
2,2000-01-05
3,2000-01-07
4,2000-01-09
...,...
839995,2082-02-09
839996,2082-02-11
839997,2082-02-13
839998,2082-02-15


In [ ]:
CT_Pat_new2 = CT_Pat_new.copy(deep=True)
CT_Pat_new2['Time'] = days_df
CT_Pat_new2

In [ ]:
PD_Pat_new2 = PD_Pat_new.copy(deep=True)
PD_Pat_new2['Time'] = days_df
PD_Pat_new2

In [ ]:
#Comprobacion 
comprobacion3 = PD_Pat_new2[(PD_Pat_new2["Patient"] == "PD4_3")].reset_index(drop=True)
comprobacion3

## Guardar tabla final para SDV ->

In [52]:
To_SDV = pd.concat([CT_Pat_new2,PD_Pat_new2],axis=0).reset_index(drop=True)
To_SDV

,Patient,Diagnosis,Time,Fp1,Fz,F3,F7,FT9,FC5,FC1,...,TP8,C6,C2,FC4,FT8,F6,AF8,AF4,F2,FCz
0,CT0_1,0,2000-01-01,156.201582,49.479165,50.302581,20.215555,-12.397837,20.148683,24.841883,...,1.185870,9.194933,17.740672,30.431134,11.963110,57.172340,111.356813,103.314326,52.204285,29.656543
1,CT0_1,0,2000-01-03,145.478530,43.666129,44.632040,16.021804,-13.106574,16.419699,20.031738,...,-0.411366,7.823835,15.794271,25.554302,8.057780,50.382441,102.441174,94.273764,45.828406,25.188870
2,CT0_1,0,2000-01-05,134.240279,37.801224,39.030074,12.405371,-13.858001,11.344561,15.244014,...,-1.502077,7.044750,13.798539,20.942617,4.690705,44.305900,93.627099,85.235874,39.671471,20.653338
3,CT0_1,0,2000-01-07,122.710755,32.325428,33.950823,9.609347,-14.912230,5.861661,11.085431,...,-1.940335,6.441215,11.988691,17.224940,2.379905,39.250948,85.051264,76.559779,34.187564,16.528426
4,CT0_1,0,2000-01-09,111.328470,27.634802,29.872545,7.843088,-15.983868,1.339396,8.125022,...,-1.900270,5.344115,10.518710,14.638750,1.456475,35.193185,76.820716,68.583172,29.692820,13.235973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679995,PD13_4,1,2082-02-09,-27.601376,-8.085939,-12.653539,-9.337065,-4.143293,-5.174250,-2.799404,...,-10.013405,-1.594446,2.354832,-9.902087,-21.483500,-22.557893,-22.842277,-19.600852,-7.616216,-2.617241
1679996,PD13_4,1,2082-02-11,-23.863473,-7.876375,-12.465237,-9.183182,-4.023284,-4.183411,-2.375053,...,-11.162566,-2.536809,2.688374,-10.238493,-21.406980,-22.375218,-23.489043,-17.534417,-7.530872,-2.160721
1679997,PD13_4,1,2082-02-13,-21.127530,-8.164021,-12.039869,-8.784472,-4.035305,-2.738989,-2.181428,...,-10.732712,-3.774637,2.709855,-11.095011,-21.153143,-22.000943,-23.123911,-16.986932,-7.604879,-1.845809
1679998,PD13_4,1,2082-02-15,-20.314355,-8.621625,-11.352318,-8.057394,-4.232272,-1.107647,-2.071666,...,-8.871847,-4.854902,2.566739,-11.980336,-20.691105,-21.440498,-21.922779,-17.846872,-7.584893,-1.585936


In [53]:
To_SDV.to_csv('%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\segmented_tables\\all_CT_PD_30s_63ch.csv' %(disco),index=False)